# Imports

In [1]:
#Imports
%matplotlib widget
import pandas as pd
import numpy as np
import csv
import matplotlib
import matplotlib.pyplot as plt
import scipy
import datetime
import sys
import networkx as nx
import misc
import math
import random
import time
import SmallBucket
import BigBucket
import RandomWarmUp1
import RandomWarmUp2
import StaticDynamic
import DcOrient
import DcOrientRandomSimple
import DcSimple
import StaticSimple

# Data Generation

In [2]:
G = misc.createRandomGraph(size=100, variation=0, density=0.5)
misc.graphInfo(G)

avg = misc.getAverageDegree(G)
variance = 0
degreeTuples = list(G.degree(list(G.nodes())))

for tuple in degreeTuples:
    variance += abs(avg - tuple[1])

variance = float(variance) / G.number_of_nodes()

variance

Nodes: 100
Edges: 2475
Density: 0.5
Max Degree: 60
Average Degree: 49.5
Static Coloring uses 20 colors


3.94

In [3]:
# EITHER
updates = misc.extractUpdates(G, 'random')

Ge = nx.Graph()
Ge.add_nodes_from(list(G.nodes()))

In [118]:
# OR
updates = misc.extractStream(G, 10000, addProb=0.5, ordering='timeout')

Ge = G.copy()

In [4]:
# Ge = nx.Graph()
# Ge.add_nodes_from(list(G.nodes()))

# Algorithm Preparation

In [6]:
experimentList = []

experimentList.append([RandomWarmUp1.WarmUp1Algo(Ge, maxDegreeBound=None), 'Random Warm-Up 1'])
experimentList.append([RandomWarmUp2.WarmUp2Algo(Ge, maxDegreeBound=None), 'Random Warm-Up 2'])

d = 10
experimentList.append([SmallBucket.SmallBucketAlgo(d, Ge), 'Small-Bucket algorithm (d = '+str(d)+')', 10])

d = 10
experimentList.append([BigBucket.BigBucketAlgo(d, Ge), 'Big-Bucket algorithm (d = '+str(d)+')', 10])

l = 10
experimentList.append([StaticDynamic.StaticDynamicAlgo(Ge, l, dynamicReset=False), 'Static-Dynamic algorithm (l = '+str(l)+')'])

l = 50
experimentList.append([StaticDynamic.StaticDynamicAlgo(Ge, l, dynamicReset=False), 'Static-Dynamic algorithm (l = '+str(l)+')'])

l = 5
experimentList.append([StaticSimple.StaticSimpleAlgo(Ge, l=l), 'Static-Simple (l = '+str(l)+')'])

l = 50
experimentList.append([StaticSimple.StaticSimpleAlgo(Ge, l=l), 'Static-Simple (l = '+str(l)+')'])

p=0.8
experimentList.append([DcOrientRandomSimple.DcOrientRandomSimpleAlgo(Ge, p=p), 'DC-Orient-Random-Simple (p = '+str(p)+')'])

p=0.8
experimentList.append([DcSimple.DcSimpleAlgo(Ge, p=p), 'DC-Simple (p = '+str(p)+')'])

experimentList.append([DcOrient.DcOrientAlgo(Ge), 'DC-Orient'])


# Experimenting

In [7]:
# CSV Writer
loadingBar = True
f = open('experiment_output.csv', 'w', newline='')
writer = csv.writer(f)

print('Graph used in the experiment:')
misc.graphInfo(G)

row = [str(G.number_of_nodes()) + ' Nodes; ' + str(G.number_of_edges()) + ' Edges; C = ' + str(misc.numberOfColors(nx.coloring.greedy_color(G))),'Average nr. of Recolors', 'Total nr. of Colors', 'Time Taken (s)']
writer.writerow(row)

for experiment in experimentList:

    row = [str(experiment[1])]

    if len(experiment) == 3:
        random.seed(experiment[2])
    else:
        random.seed()

    print("")
    print("------------------------------------------------------------------")
    print(str(experiment[1]) + ":")
    iterator = misc.UpdateIterator(experiment[0], updates)

    recolors = 0
    timer = time.perf_counter()

    totalUpdates = len(updates)
    done = 0
    barLength = 64

    if loadingBar:
        #line to be overwritten
        print("")

    while True:
        
        if loadingBar:
            #overwrite previously printed line
            percentageDone = int(done/totalUpdates*barLength)
            b = ("[" + percentageDone * "+" + (barLength - percentageDone) * "-" + "]")
            sys.stdout.write('\r'+b)
            sys.stdout.flush()

        if not misc.verifyColoring(iterator.algo.G, iterator.algo.getColoring()):
            raise ValueError('The coloring produced is incorrect')
        c1 = iterator.algo.getColoring()
        if iterator.runUpdate(1):
            c2 = iterator.algo.getColoring()
            recolors += misc.numberOfRecolors(c1, c2)
            done += 1
        else:
            # Verify coloring one last time before ending this experiment
            if not misc.verifyColoring(iterator.algo.G, iterator.algo.getColoring()):
                raise ValueError('The coloring produced is incorrect')
            #print("all updates finished")
            break

    timer = time.perf_counter() - timer

    if loadingBar:
        #overwrite previously printed line
        b = ("")
        sys.stdout.write('\r'+b)
        sys.stdout.flush()

    avgColors = float(recolors)/len(updates)
    b = "Average number of recolors per update: " + str(avgColors)
    b = b + (barLength + 2 - len(b)) * " "
    print(b)
    row.append(str(round(avgColors, 2)))
    totColors = misc.numberOfColors(iterator.algo.getColoring())
    print("Total number of colors used at the end: " + str(totColors))
    row.append(str(totColors))
    print("Time taken : " + str(timer) + " seconds")
    row.append(str(round(timer, 3)))
    print("Total time spent on accessing neighbors : " + str(iterator.elemCounter))
    #print("Average time spent on accessing neighbors : " + str(iterator.avgElemCounter))
    print("------------------------------------------------------------------")
    writer.writerow(row)

print("End of experiment list reached")
f.close()


Graph used in the experiment:
Nodes: 100
Edges: 2475
Density: 0.5
Max Degree: 60
Average Degree: 49.5
Static Coloring uses 20 colors

------------------------------------------------------------------
Random Warm-Up 1:

Average number of recolors per update: 0.11070707070707071        
Total number of colors used at the end: 45
Time taken : 3.3922798000000043 seconds
Total time spent on accessing neighbors : 0.005860700000006602
------------------------------------------------------------------

------------------------------------------------------------------
Random Warm-Up 2:

Average number of recolors per update: 0.17616161616161616        
Total number of colors used at the end: 43
Time taken : 3.2335244000000003 seconds
Total time spent on accessing neighbors : 0.012315500000035229
------------------------------------------------------------------

------------------------------------------------------------------
Small-Bucket algorithm (d = 10):

Average number of recolors per 

# Results